# Neighbourhood Essential Supplies:
## Week 1:
 ### Description:
 In light of COVID-19 going forward there will be an ever greater demand for social distanced shopping regardless of the disease prvelance as people have now become aware of the direct exposure to ilness caused by grouping in crowded places. Second to this increased work from home rates pent up demand, goverment support and an ever growing demand for popular main stream items and shopping experiences. This all leads to a greater demand for housing in neghbourhoods with a wide variety of popular shopping destinations nearby. This allows one to easy choose between different locations based on the crowdedness of each location.
 Furthermore certain individuals have different needs as such may choose there next housing location based on these needs. For this I will peform 3 different clustering teqcniques to find the best location based on their needs: The first will be based on only essential locations the second will compare supporting locations such as restuarants, coffee shops and other retail stores.
 As a proxy for the desirability of the location I will only be using the top 10 locations near each area.
The relevance of either location will be based on the individuals covid risk. High risk individuals should prefer locations with a high number of popular essential locations to allow for a choice in less crowded establishments.

 ### Data:
 I will be making use of the Toronto neibourghood data gathered in previous assignments and determine the number of essential and the popular lockdown prooof locations,namely: Pharmac....., in each neiohbourhood, I will then cluster the neihbourhood's based on the number of popular essential establishments in the neibourghoods.I will then peform an analysis to determine the most appropriate neighbourhoods for individuals based on there COVID-19 risk. 

## Week 2: Full Report 

### Introduction: 
In light of COVID-19 going forward there will be an ever greater demand for social distanced shopping regardless of the disease prvelance as people have now become aware of the direct exposure to ilness caused by grouping in crowded places. Second to this increased work from home rates pent up demand, goverment support and an ever growing demand for popular main stream items and shopping experiences. This all leads to a greater demand for housing in neghbourhoods with a wide variety of popular shopping destinations nearby. 

Neghbourhood planners, investors, goverment officals, people in the property market and those at risk to seevere COVID-19 may all find use in this data. Not only will it allow for better forward planning and greater insights for future property investments but it could help a great deal of people to drive to locations where they are best able to have there needs met whilst still being able to choose the less crowded establishment.

### Data:
Data Used: Tornonto,Canada Neighbourhood Data;Venue data for essential shopping locations+Longitude/Latitude data for each respective location.
Data Source: 4Square API:Location data , Geocoder package:Geolocation data.

### Methadology:

The first step of the process will be installing the required packages:


In [2]:
pip install geocoder

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 98 kB 9.8 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install folium

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 1.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Importing packages 
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import folium
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

#### We need to import Toronto location data:

In [5]:
#Importing data:
##Toronto:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(source, 'lxml')

table = soup.find('table')
table_rows = table.tbody.find_all('tr')

## Creating the pandas data frame:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [6]:
df_geo_coor=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")
df_geo_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### We now need to pair the Postal Codes to their Lat/longs:¶


In [7]:
df_LL=pd.merge(df,df_geo_coor,left_on="PostalCode",right_on="Postal Code")
df_LL.head()

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,M7A,43.662301,-79.389494


### Map of Tornoto and its Neighborhoods:

In [8]:
# Finding the mean point of all Neighborhoods so that Map is centered:
mean_lat=df_LL.Latitude.mean()
mean_long=df_LL.Longitude.mean()
print(mean_lat)
print(mean_long)

###
longitude = mean_long
latitude = mean_lat
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map based on coordinates of postcode
for lat, lng, Neighbourhood in zip(df_LL['Latitude'], df_LL['Longitude'], df_LL['Neighborhood']):
    label = '{}'.format(Neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

43.704607733980595
-79.3971529116505


#### 4Square API:

In [70]:
# Log into Foursquare:

CLIENT_ID = 'BQCYU4IEPQXQ005Y4HAYT4QWPWRKBTKGLFRNV4HSKFMPIZDN' # your Foursquare ID
CLIENT_SECRET = '0K12WQKYXZB3PD4SMM2DORTYKOKOIU34ZTW3EQF0LN3EG4Q3' # your Foursquare Secret
VERSION = '20161225' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BQCYU4IEPQXQ005Y4HAYT4QWPWRKBTKGLFRNV4HSKFMPIZDN
CLIENT_SECRET:0K12WQKYXZB3PD4SMM2DORTYKOKOIU34ZTW3EQF0LN3EG4Q3


In [275]:
# define inputs:
# Number of locations to be displayed:
Locations = 10
# Distance from neighbourhood to be included:
distance = 500

locations = [] # initiate a list to store data from Foursquare API requests

for neighbourhood, latitude, longitude in zip(df_LL.Neighborhood, df_LL.Latitude, df_LL.Longitude):
    
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    distance, 
    Locations)
    
    data = requests.get(url).json()
    
    # use len() to check if any data within "items" (len = 0 if nothing)
    length = len(data['response']['groups'][0]['items'])
    if length == 0:
        continue # skip the row if nothing is found
    else:
        venue = data['response']['groups'][0]['items'][0]['venue']
        
        # extract info within 'venue'
        name = venue['name']
        lat = venue['location']['lat']
        lon = venue['location']['lng']
        cat = venue['categories'][0]['name']
        
        locations.append([(neighbourhood, latitude, longitude, name, lat, lon, cat)])


In [276]:
temp = pd.DataFrame(x for row in locations for x in row)
temp.columns = ['Neighborhood','N_Latitude','N_Longitude','Venue','V_Latitude','V_Longitude','category']
temp.head()
print("{} nearby locations downloaded for {} neighborhood.".format(len(temp.Venue), len(df_LL.Neighborhood)))

98 nearby locations downloaded for 103 neighborhood.


We now need to find the frequency of each category around each location: Using get_dummies to create a matrix of location and category freqency: 

In [277]:
category_freqency = pd.get_dummies(temp.category) # one hot encoding with get_dummies()
Neigh_Cat_Freq = pd.concat([temp[['Neighborhood']], category_freqency], axis=1) # combine neighbourhood & category tables
Neigh_Cat_Freq.head()

,Neighborhood,Airport,Arts & Crafts Store,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,...,Sandwich Place,Skating Rink,Sports Bar,Supermarket,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Warehouse Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Lawrence Manor, Lawrence Heights",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ontario Provincial Government,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### We will now create 3 tables the first containing only essential establishments the second containing only support establishments and the third will contain both the contents of the first and 2nd tables.

In [278]:
Neigh_Cat_Freq=Neigh_Cat_Freq.squeeze()
Neigh_Cat_Freq.head()

,Neighborhood,Airport,Arts & Crafts Store,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,...,Sandwich Place,Skating Rink,Sports Bar,Supermarket,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Warehouse Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Lawrence Manor, Lawrence Heights",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ontario Provincial Government,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [285]:
len(Neigh_Cat_Freq.columns)

60

In [286]:
Neigh_Cat_Freq_comb=Neigh_Cat_Freq
print(len(Neigh_Cat_Freq_comb.columns))
(Neigh_Cat_Freq_comb.head())

60


,Neighborhood,Airport,Arts & Crafts Store,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,...,Sandwich Place,Skating Rink,Sports Bar,Supermarket,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Warehouse Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Lawrence Manor, Lawrence Heights",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ontario Provincial Government,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [287]:
Essential= ['Airport','Bank','Convenience Store','Fast Food Restaurant', 'Fish & Chips Shop', 'Food & Drink Shop','Grocery Store','Pharmacy', 'Supermarket','Warehouse Store', 'Clothing Store','Deli / Bodega']
Neigh_Cat_Freq_support=Neigh_Cat_Freq
for i1 in Essential:
    try:
       Neigh_Cat_Freq_support=Neigh_Cat_Freq_support.drop(i1, axis=1)
    except:
        pass
print(len(Neigh_Cat_Freq_support.columns))
(Neigh_Cat_Freq_support.head())

48


,Neighborhood,Arts & Crafts Store,Bakery,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,Burger Joint,Café,...,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Sports Bar,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Lawrence Manor, Lawrence Heights",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ontario Provincial Government,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [288]:
Support= ['Arts & Crafts Store', 'Bakery','Bar', 'Baseball Field', 'Boutique', 'Breakfast Spot', 'Brewery','Café', 'Chinese Restaurant', 'Coffee Shop','Cosmetics Shop', 'Creperie','Department Store', 'Dessert Shop', 'Dog Run', 'Fast Food Restaurant','Field', 'Fish & Chips Shop', 'Gastropub','Gym', 'Hakka Restaurant', 'Hockey Arena','Home Service', 'Hotel', 'Ice Cream Shop', 'Intersection','Italian Restaurant', 'Japanese Restaurant', 'Jewelry Store','Liquor Store', 'Massage Studio', 'Motel', 'Park','Piano Bar', 'Pizza Place', 'Playground', 'Plaza','Rental Car Location', 'Restaurant', 'River', 'Sandwich Place','Skating Rink', 'Sports Bar', 'Theme Restaurant','Toy / Game Store', 'Trail', 'Vegetarian / Vegan Restaurant','Wings Joint', 'Yoga Studio','Burger Joint']
Neigh_Cat_Freq_essential=Neigh_Cat_Freq
for i2 in Support:
    try:
      Neigh_Cat_Freq_essential=Neigh_Cat_Freq_essential.drop(i2, axis=1)
    except:
        pass
print(len(Neigh_Cat_Freq_essential.columns))
(Neigh_Cat_Freq_essential.head())

16


,Neighborhood,Airport,Bank,Clothing Store,Construction & Landscaping,Convenience Store,Deli / Bodega,Food & Drink Shop,Furniture / Home Store,Garden,Grocery Store,Neighborhood,Pharmacy,Pool,Supermarket,Warehouse Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Lawrence Manor, Lawrence Heights",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ontario Provincial Government,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [289]:
#Storing the dataframes in more concise variable names:
df_Ess=Neigh_Cat_Freq_essential
df_Supp=Neigh_Cat_Freq_support
df_Comb=Neigh_Cat_Freq_comb

In [290]:
df_Comb.head()

,Neighborhood,Airport,Arts & Crafts Store,Bakery,Bank,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,...,Sandwich Place,Skating Rink,Sports Bar,Supermarket,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Warehouse Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Lawrence Manor, Lawrence Heights",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ontario Provincial Government,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [291]:
df_Supp.head()

,Neighborhood,Arts & Crafts Store,Bakery,Bar,Baseball Field,Boutique,Breakfast Spot,Brewery,Burger Joint,Café,...,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Sports Bar,Theme Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Lawrence Manor, Lawrence Heights",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Ontario Provincial Government,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [292]:
df_Ess.head()

,Neighborhood,Airport,Bank,Clothing Store,Construction & Landscaping,Convenience Store,Deli / Bodega,Food & Drink Shop,Furniture / Home Store,Garden,Grocery Store,Neighborhood,Pharmacy,Pool,Supermarket,Warehouse Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Lawrence Manor, Lawrence Heights",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ontario Provincial Government,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Analysis:
As we have now split up the data into Essenrtial, Support and Combined, I will now go through analysising the diffferences between the different categories and neighbourhoods.

In [293]:
# Importing the nescasery packages:
from sklearn.cluster import KMeans

In [467]:
#Inputs:
## Number of clusters:
k=3
#################################
##Input: 
df=df_Comb

## The input for kMeans needs to be a matrix, we therefore need to remove the first column that contains the Neighborhoods name.
df = df.drop('Neighborhood', axis=1)
# run k-means clustering
kmeans = KMeans(n_clusters=6, random_state=0).fit(df)
kmeans.labels_[0:10]

## Output:
kmeans_Comb=kmeans
Comb_lable=kmeans.labels_[0:10]
try:
    df_Comb.insert(1, 'label', kmeans_Comb.labels_)
except:
    pass
#################################
#################################
##Input: 
df=df_Ess

## The input for kMeans needs to be a matrix, we therefore need to remove the first column that contains the Neighborhoods name.
df = df.drop('Neighborhood', axis=1)
# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(df)
kmeans.labels_[0:10]

## Output:
kmeans_Ess=kmeans
Ess_lable=kmeans.labels_[0:10]
try:
    df_Ess.insert(1, 'label', kmeans_Ess.labels_)
except:
    pass
#################################
#################################
##Input: 
df=df_Supp

## The input for kMeans needs to be a matrix, we therefore need to remove the first column that contains the Neighborhoods name.
df = df.drop('Neighborhood', axis=1)
# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(df)
kmeans.labels_[0:10]

## Output:
kmeans_Supp=kmeans
Supp_lable=kmeans.labels_[0:10]
try:
    df_Supp.insert(1, 'label', kmeans_Supp.labels_)
except:
    pass
#################################
print(Supp_lable,Ess_lable,Comb_lable)

[1 0 0 0 1 0 2 0 0 0] [0 0 0 0 0 0 0 0 0 0] [1 0 0 0 1 0 4 0 0 0]


In [480]:
# Removing duplicate column headings: Comb
df1 = df_Comb.loc[:, ~df_Comb.columns.duplicated()]
#Combining the geolocation data(df_LL) with category data (df_Comb)
t_comb=pd.merge(df_LL,df1, on ='Neighborhood',how ='left' )
t_comb=t_comb.fillna(0)

# Removing duplicate column headings: Ess
df1 = df_Ess.loc[:, ~df_Ess.columns.duplicated()]
#Combining the geolocation data(df_LL) with category data (df_Comb)
t_ess=pd.merge(df_LL,df1, on ='Neighborhood',how ='left' )
t_ess=t_ess.fillna(0)

# Removing duplicate column headings: supp
df1 = df_Supp.loc[:, ~df_Supp.columns.duplicated()]
#Combining the geolocation data(df_LL) with category data (df_Comb)
t_supp=pd.merge(df_LL,df1, on ='Neighborhood',how ='left' )
t_supp=t_supp.fillna(0)





In [481]:
#for lat, lon, poi, cluster in zip(t_comb['Latitude'], t_comb['Longitude'], t_comb['Neighborhood'], t_comb['label']):
  #  print(cluster)
    
print(max(t_comb['label']),min(t_comb['label']))

5.0 0.0


In [482]:
Col=['red', 'blue', 'green', 'purple', 'orange', 'darkred','lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

In [483]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)
#Input table: 
# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(t_comb['Latitude'], t_comb['Longitude'], t_comb['Neighborhood'], t_comb['label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        cluster=1,
        color=Col[(int(cluster))],
        fill=True,
        fill_color=Col[(int(cluster))],
        fill_opacity=0.7).add_to(map_clusters)

folium.LayerControl(position='topright', collapsed=True, autoZIndex=True).add_to(map_clusters)

Comb_Map=map_clusters

In [484]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)
#Input table: 

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(t_ess['Latitude'], t_ess['Longitude'], t_ess['Neighborhood'], t_ess['label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        cluster=1,
        color=Col[(int(cluster))],
        fill=True,
        fill_color=Col[(int(cluster))],
        fill_opacity=0.7).add_to(map_clusters)

folium.LayerControl(position='topright', collapsed=True, autoZIndex=True).add_to(map_clusters)

Ess_Map=map_clusters

In [485]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)
#Input table: 
# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(t_supp['Latitude'], t_supp['Longitude'], t_supp['Neighborhood'], t_supp['label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        cluster=1,
        color=Col[(int(cluster))],
        fill=True,
        fill_color=Col[(int(cluster))],
        fill_opacity=0.7).add_to(map_clusters)

folium.LayerControl(position='topright', collapsed=True, autoZIndex=True).add_to(map_clusters)

Supp_Map=map_clusters

In [486]:
t_supp['label']==t_comb['label']

0      False
1      False
2      False
3      False
4      False
       ...  
98     False
99     False
100    False
101    False
102    False
Name: label, Length: 103, dtype: bool

# Results: 
Now that we have clustered the 3 different categories I will now display and analyse the results.


### Combined:

In [487]:
Comb_Map

### Seperate:
Supporting(Left) and Essential(Right) Establishments:

In [489]:
from IPython.core.display import display, HTML

htmlmap = HTML('<iframe srcdoc="{}" style="float:left; width: {}px; height: {}px; display:inline-block; width: 50%; margin: 0 auto; border: 2px solid black"></iframe>'
           '<iframe srcdoc="{}" style="float:right; width: {}px; height: {}px; display:inline-block; width: 50%; margin: 0 auto; border: 2px solid black"></iframe>'
           .format(Supp_Map.get_root().render().replace('"', '&quot;'),500,500,
                   Ess_Map.get_root().render().replace('"', '&quot;'),500,500))
display(htmlmap)

The results of the anlysis indicate that there are clear differences between the avalability of Essential and Supporting establishments in the Neighborhoods in and around Tornonto: 

Combined: Looking at the combined map there appears to be no desrnable trend as the clusters appear to be evenly disperesed amongst the Neighhborhoods. This indicates that purely looking at the available establshments may lead to persons moving to areas that my not meet there needs, from a city planner or investors perspective thise may mean lead to a less favourable outcome as the new establishment may be erected in a neighborhood that already has these needs met.

Supporting Establishments:
There seems to be some differences in the availablity of non-essential establishments. Clusters appear to change based on their proximety to Tornoto city center. This indicates that the amount of Supporting Establishments increases the closer to the city we are. For persons who's needs are otherwise met and thus would prefer living in an area with adequate supporting establishments this information suggests that they would be best off living near the city center. 

Essential Establishments: 
Again there seems to be some very interesting data contaioned in this cluster map. It is clear that the access to essential establishments is limited in and around the city center. There appears to be a greater level of concentraition amoungst the clusters. This means that even though there are less clusters with estabishments that meet the criteria, the clusters that do meet the criteria have a higher number of estabishments in one location. Interestingly essentail establishments also seem to be placed near airports. 



# Discussion:
The results of this analysis provided conclusive evidence to answer the initial buisness questions. 
The most clear of this is the identification of locations that have a high number of essential lestablishments in there facinity, these are represented in blue on the map. This means that for high risk individuals I would suggest in staying in such locations as on the occasion that one must go to the shops, such areas allow for greater choice amonst popular establishments. Whilst for those not at risk they may find more benefit living in and around the city so that they can take advantage of the greater provison of supporting establishments.

Secondly was the dispersion of supporting establishments in Torono, many of the supporting locations are in and around the city center. Whilst this may have made sence pre covid-19, post covid-19, with a greater number of people working from home it can be expected that a greater number of people will be working from home and thus spend less time in the city.This means that going forward there may be greter room for investemn t in supporting establishments outside of the city center and probably time for de-inevestment in the city center. 

From a City planer and goverment officals perspective the Essential Estabishment map should be of slight concern, certain areas lack access to essential needs, this could lead to overcorwding in neighboring establishments or long distances traveled.

# Conclusion: 
COVID-19 has changed alot about what certain individuals demand and have access to. Older members are now more catious of establishments that are over-crowded and would avoid public spaces whereever possible. Similiarly for the younger generations, work from home leads to a more restrictive daily travel routine as well as increasing the demand for supporting establishments to make up for the missed social interaction that would be had at work.

Ultimetly the differences displayed in this report may indicate signficant to modereate migration of younger individuals towards the city and older individuals towards the outer neighbourhoods and could suggest possible investment in Essential goods in and around the city center. 
